In [1]:
import re

In [45]:
%cd "C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\PATHWAY TOOLS\phaseolusVulgaris\SBML_files"
with open("Parsing_format.txt", 'r') as IN:
    lines = IN.readlines()

%cd "C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\GAMS\Parsing\Pvul_test"
SMATRIX = open('S_matrix.txt', 'a')
COMPOUNDS = open('compounds-1.txt', 'a')
RXN = open('reactions.txt', 'a')
RRXN = open('reversible_reactions.txt', 'a')
UPPBOUND = open('upperbound_on_fluxes.txt', 'a')
LOWBOUND = open('lowerbound_on_fluxes.txt', 'a')
CYTOSOLIC=open("cytosolic_compounds.txt",'a')
EXTRACELL=open("extracellular_compounds.txt","a")

SMATRIX.write("/\n")
RXN.write("/\n")
RRXN.write("/\n")
UPPBOUND.write("/\n")
LOWBOUND.write("/\n")
CYTOSOLIC.write("/\n")
EXTRACELL.write("/\n")

C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\PATHWAY TOOLS\phaseolusVulgaris\SBML_files
C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\GAMS\Parsing\Pvul_test


2

In [46]:
listCompounds = []
for line in lines:
    line = line.strip()
    fields = line.split(";")
   #if fields[3]:
        #print("compartment defined")
   #else:
        #print("compartment not defined")
    comp = fields[3].split(',')

    for c in comp:
        RXN.write(f"'{fields[0]}[{c}]'\n")
        #print(f"RXN: '{fields[0]}[{c}]'\n")

        
    # 2. Now analyze everything to generate the matrix and the list of compounds
    if "<=>" in fields[1]:
        #print("reversible reaction")
        met = fields[1].split(" <=> ")
        met_izq = met[0].split(" + ")
        met_der = met[1].split(" + ")
        #print("METDER", met_der)
    else:
        #print("irreversible reaction")
        met = fields[1].split(" --> ")
        met_izq = met[0].split(" + ")
        met_der = met[1].split(" + ")
        #print(met_izq, met_der)
    
    if "<=>" in fields[1]:
        for c in comp:
            RRXN.write(f"'{fields[0]}[{c}]'\n")
            #print(f"RRXN: '{fields[0]}[{c}]'\n")

        for met_izq_2 in met_izq:
            if len(met_izq_2.split())>1:
                coef=met_izq_2.split()[0]   # If there is a coefficient
                compound = met_izq_2.split()[1]
                for c in comp:
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[{c}]' -{coef}\n")
                    #print(f"COMPOUNDS_IZQ: '{compound}[{c}]\n'")
                    #print(f"SMATRIX_IZQ: '{compound}[{c}]'.'{fields[0]}[c]' -{coef}\n")
                    if (f"'{compound}{[c]}'") not in listCompounds:
                        listCompounds.append(f"'{compound}{[c]}'")
                        COMPOUNDS.write(f"'{compound}[{c}]'\n")
                        
            else:
                coef = 1
                compound = met_izq_2.split()[0]
                SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[{c}]' -1\n")
                for c in comp:
                    #print(f"COMPOUNDS_IZQ: '{compound}[{c}]\n'")
                    #print(f"SMATRIX_IZQ: '{compound}[{c}]'.'{fields[0]}[c]' -1\n")
                    if (f"'{compound}{[c]}'") not in listCompounds:
                        listCompounds.append(f"'{compound}{[c]}'")
                        COMPOUNDS.write(f"'{compound}[{c}]'\n")
                        

        for met_der_2 in met_der:
            if len(met_der_2.split()) >1:
                coef=met_der_2.split()[0]   # If there is a coefficient
                compound = met_der_2.split()[1]
                for c in comp:
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[{c}]' {coef}\n")
                    #print(f"COMPOUNDS_IZQ: '{compound}[{c}]\n'")
                    #print(f"SMATRIX_IZQ: '{compound}[{c}]'.'{fields[0]}[c]' -{coef}\n")
                    if (f"'{compound}{[c]}'") not in listCompounds:
                        listCompounds.append(f"'{compound}{[c]}'")
                        COMPOUNDS.write(f"'{compound}[{c}]'\n")
                        
            else:
                coef = 1
                compound = met_der_2.split()[0]
                for c in comp:
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[{c}]' 1\n")
                    #print(f"COMPOUNDS_IZQ: '{compound}[{c}]\n'")
                    #print(f"SMATRIX_IZQ: '{compound}[{c}]'.'{fields[0]}[c]' -1\n")
                    if (f"'{compound}{[c]}'") not in listCompounds:
                        listCompounds.append(f"'{compound}{[c]}'")
                        COMPOUNDS.write(f"'{compound}[{c}]'\n")
                        

        # Printing reaction limits
        for c in comp:
            #print(f"'{fields[0]}[{c}]' 1000\n")
            #print(f"'{fields[0]}[{c}]' -1000\n")
            UPPBOUND.write(f"'{fields[0]}[{c}]' 1000\n")
            LOWBOUND.write(f"'{fields[0]}[{c}]' -1000\n")
        
    elif "-->" in fields[1]:  # Evaluating Forward
        for met_izq_2 in met_izq:
            if len(met_izq_2.split())>1:
                coef=met_izq_2.split()[0]   # If there is a coefficient
                compound = met_izq_2.split()[1]
                SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[c]' -{coef}\n")
                for c in comp:
                    if (f"'{compound}{[c]}'") not in listCompounds:
                        listCompounds.append(f"'{compound}{[c]}'")
                        COMPOUNDS.write(f"'{compound}[{c}]'\n")
                        
            else:
                coef = 1
                compound = met_izq_2.split()[0]
                for c in comp:
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[{c}]' -1\n")
                    #print(f"COMPOUNDS_IZQ: '{compound}[{c}]\n'")
                    #print(f"SMATRIX_IZQ: '{compound}[{c}]'.'{fields[0]}[c]' -1\n")
                    if (f"'{compound}{[c]}'") not in listCompounds:
                        listCompounds.append(f"'{compound}{[c]}'")
                        COMPOUNDS.write(f"'{compound}[{c}]'\n")
                        

        for met_der_2 in met_der:
            if len(met_der_2.split()) >1:
                coef=met_der_2.split()[0]   # If there is a coefficient
                compound = met_der_2.split()[1]
                SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[c]' {coef}\n")
                for c in comp:
                    if (f"'{compound}{[c]}'") not in listCompounds:
                        listCompounds.append(f"'{compound}{[c]}'")
                        COMPOUNDS.write(f"'{compound}[{c}]'\n")
                        
            else:
                coef = 1
                compound = met_der_2.split()[0]
                for c in comp:
                    SMATRIX.write(f"'{compound}[{c}]'.'{fields[0]}[{c}]' 1\n")
                    #print(f"COMPOUNDS_IZQ: '{compound}[{c}]\n'")
                    #print(f"SMATRIX_IZQ: '{compound}[{c}]'.'{fields[0]}[c]' -1\n")
                    if (f"'{compound}{[c]}'") not in listCompounds:
                        listCompounds.append(f"'{compound}{[c]}'")
                        COMPOUNDS.write(f"'{compound}[{c}]'\n")

        for c in comp:
            #print(f"'{fields[0]}[{c}]' 1000\n")
            #print(f"'{fields[0]}[{c}]' 0\n")
            UPPBOUND.write(f"'{fields[0]}[{c}]' 1000\n")
            LOWBOUND.write(f"'{fields[0]}[{c}]' 0\n")

for met in listCompounds:
    if "[c]" in met:
        CYTOSOLIC.write(f"{met}\n")
    elif "[e]" in met:
        EXTRACELL.write(f"{met}\n")

In [47]:
SMATRIX.write("/\n")
RXN.write("/\n")
RRXN.write("/\n")
UPPBOUND.write("/\n")
LOWBOUND.write("/\n")
CYTOSOLIC.write("/\n")
EXTRACELL.write("/\n")


SMATRIX.close()
COMPOUNDS.close()
RXN.close()
RRXN.close()
UPPBOUND.close()
LOWBOUND.close()   
CYTOSOLIC.close()
EXTRACELL.close()